# Step 1: Preparing a Dataset with Embeddings

Add your API key to the cell below then run it.

In [1]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows")

print("Path to dataset files:", path)


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\100775689\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 175k/175k [00:00<00:00, 8.77MB/s]

Extracting files...
Path to dataset files: C:\Users\100775689\.cache\kagglehub\datasets\harshitshankhdhar\imdb-dataset-of-top-1000-movies-and-tv-shows\versions\1


In [4]:
import os

# List files in the directory
files = os.listdir(path)
print(files)

import pandas as pd
# Load the dataset into a DataFrame
df = pd.read_csv(os.path.join(path, 'imdb_top_1000.csv'))
# Display the first few rows of the DataFrame
df.head()

['imdb_top_1000.csv']


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [14]:
#load env file
!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()
# Load environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install openai==0.26.1
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = OPENAI_API_KEY


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creating the Embeddings Index

Here is the text from the first row of our dataset. Run the cell below as-is.

This code creates embeddings for that text sample. Run the cell below as-is.

In [13]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
response = openai.Embedding.create(
    input=[df["Overview"][0]],
    engine=EMBEDDING_MODEL_NAME
)

# Extract and print the first 20 numbers in the embedding
response_list = response["data"]
first_item = response_list[0]
first_item_embedding = first_item["embedding"]
print(first_item_embedding[:20])

[0.02355373278260231, -0.03621044009923935, 0.012330841273069382, -0.051226433366537094, -0.017362242564558983, 0.015224548056721687, 0.0222632959485054, -0.006155644543468952, -0.02498755045235157, -0.01087747048586607, 0.006914916913956404, 0.008713707327842712, 0.007664413191378117, 0.004145039711147547, 0.011842039413750172, 0.004555633291602135, 0.03756605088710785, -0.012611087411642075, 0.011346720159053802, -0.012265668250620365]


### Creating a list of embeddings

This code sends all of the data from `df["text"].tolist()` to the `openai.Embedding.create` function, then extracts the resulting embeddings and creates a list of embeddings called `embeddings`.

Run the cell below as-is.

In [11]:
df["Overview"][0]

'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

In [14]:
# Send text data to the model
response = openai.Embedding.create(
    input=df["Overview"].tolist(),
    engine=EMBEDDING_MODEL_NAME
)

# Extract embeddings
embeddings = [data["embedding"] for data in response["data"]]

### Adding Embeddings to DataFrame and Saving as CSV

Run the cell below as-is.

In [15]:
# Add embeddings list to dataframe
df["embeddings"] = embeddings
df.to_csv("embeddings.csv")

## Conclusion

You have now created and saved an embeddings index!

In [2]:
import pandas as pd
# Load the dataset into a DataFrame
df = pd.read_csv("embeddings.csv")
# Display the first few rows of the DataFrame
df.head()

,Unnamed: 0,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,embeddings
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469","[0.02355373278260231, -0.03621044009923935, 0...."
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","[0.00454982602968812, -0.03938516229391098, 0...."
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","[-0.0033839927054941654, -0.03664616495370865,..."
3,3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","[-0.0003739971143659204, -0.04081740230321884,..."
4,4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","[-0.020591681823134422, -0.02479379065334797, ..."


# Step 2: Finding Relevant Data

Add your API key to the cell below then run it.

In [3]:
!pip install openai==0.26.1
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = OPENAI_API_KEY


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


The code below loads in the embeddings you previously created. Run it as-is.

In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,embeddings
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469","[0.02355373278260231, -0.03621044009923935, 0...."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","[0.00454982602968812, -0.03938516229391098, 0...."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","[-0.0033839927054941654, -0.03664616495370865,..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","[-0.0003739971143659204, -0.04081740230321884,..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","[-0.020591681823134422, -0.02479379065334797, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,"[-0.01512514054775238, -0.020166853442788124, ..."
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,"[-0.011792005971074104, -0.008015654049813747,..."
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","[-0.03126581758260727, -0.028749875724315643, ..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,"[0.00025537292822264135, -0.0426042377948761, ..."


## TODO 1: Create Embeddings for the User's Question

In the previous exercise, you were given the code to create embeddings for everything in the dataset. Now your task is to create embeddings for just one string: the user's question. Assign the result to the variable `question_embeddings`. This variable should contain a list of 1,536 floating point numbers, and the provided code will print the first 100 once `question_embeddings` has been created correctly.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """What were the estimated damages of the 2023 \
Turkey-Syria earthquake?"""

# Generate the embedding response
response = openai.Embedding.create(
    input=USER_QUESTION,
    engine=EMBEDDING_MODEL_NAME
)

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

print(question_embeddings[:100])
```

</details>

In [10]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """A man is stuck in a dream world where time moves differently, and he has to plant an idea in someone's mind."""

# Generate the embedding response
response = openai.Embedding.create(
    input=USER_QUESTION,
    engine=EMBEDDING_MODEL_NAME
)

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

print(question_embeddings[:100])


[-0.0035816426388919353, -0.03116384893655777, 0.0025640937965363264, -0.014650115743279457, 0.007998031564056873, 0.014481872320175171, -0.023230528458952904, -0.004762581549584866, -0.029636718332767487, 0.0041802008636295795, 0.002439528936520219, 0.012799438089132309, 0.010230490937829018, -0.0005848074797540903, 0.021315140649676323, -0.0146371740847826, 0.020926887169480324, 0.003711060620844364, 0.013265343382954597, -0.020008020102977753, -0.01367948018014431, -0.00889748614281416, 0.016500793397426605, -0.00864512100815773, 0.0046105156652629375, 0.002274520928040147, 0.020396273583173752, -0.011777035892009735, 0.02958494983613491, -0.00429344130679965, 0.015607808716595173, -0.0001015122325043194, 0.002091718139126897, -0.015478390268981457, -0.03693589195609093, -0.018804432824254036, 0.004681695252656937, -0.015491331927478313, 0.013912432827055454, -0.01851971261203289, 0.02728131040930748, 0.006878565531224012, -0.0061991214752197266, -0.012974152341485023, -0.0149736599

## TODO 2: Find Cosine Distances

Create a new list called `distances`, which represents the cosine distances between `question_embeddings` and each value in the `'embeddings'` column of `df`.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(
    question_embeddings,
    df["embeddings"],
    distance_metric="cosine"
)

print(distances[:100])
```

</details>

In [11]:
!pip install tenacity==8.1.0
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(
    question_embeddings,
    df["embeddings"],
    distance_metric="cosine"
)

print(distances[:100])

[np.float64(0.2249443689199654), np.float64(0.25243095885019906), np.float64(0.22000951430317017), np.float64(0.22304401608285318), np.float64(0.20018682018139833), np.float64(0.23721474589092562), np.float64(0.26057688612226493), np.float64(0.23322697868153464), np.float64(0.21189075922380785), np.float64(0.20025587223799501), np.float64(0.24846566066056097), np.float64(0.2233461326285313), np.float64(0.23876827937060063), np.float64(0.22419842557058944), np.float64(0.23957063981787896), np.float64(0.21401029854691267), np.float64(0.22641000562098212), np.float64(0.21603132903722055), np.float64(0.21867679591506217), np.float64(0.23937630635996254), np.float64(0.1935139891926072), np.float64(0.22362662080085716), np.float64(0.23979271418164727), np.float64(0.22812740178977242), np.float64(0.25412729352362595), np.float64(0.2268676794297273), np.float64(0.25176878914671474), np.float64(0.22902824124859344), np.float64(0.1884835857741669), np.float64(0.1797554496526016), np.float64(0.21


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Sorting by Distance

The code below uses the `distances` list to update `df` then sorts `df` to find the most related rows. Shorter distance means more similarity, so we'll use an ascending sorting order. Run the cell below as-is.

In [12]:
df["distances"] = distances
df.sort_values(by="distances", ascending=True, inplace=True)
df.head(5)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,embeddings,distances
8,https://m.media-amazon.com/images/M/MV5BMjAxMz...,Inception,2010,UA,148 min,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,2067042,"292,576,195","[-0.005405026953667402, -0.03797874599695206, ...",0.117507
643,https://m.media-amazon.com/images/M/MV5BMWM0Zj...,Waking Life,2001,R,99 min,"Animation, Drama, Fantasy",7.8,A man shuffles through a dream meeting various...,83.0,Richard Linklater,Ethan Hawke,Trevor Jack Brooks,Lorelei Linklater,Wiley Wiggins,60684,"2,892,011","[0.00906852725893259, -0.025172190740704536, 0...",0.131207
959,https://m.media-amazon.com/images/M/MV5BMGExOG...,Dark City,1998,A,100 min,"Mystery, Sci-Fi, Thriller",7.6,"A man struggles with memories of his past, whi...",66.0,Alex Proyas,Rufus Sewell,Kiefer Sutherland,Jennifer Connelly,William Hurt,187927,"14,378,331","[0.005642868112772703, -0.008322513662278652, ...",0.143791
919,https://m.media-amazon.com/images/M/MV5BMjkyMT...,Stardust,2007,U,127 min,"Adventure, Family, Fantasy",7.6,In a countryside town bordering on a magical l...,66.0,Matthew Vaughn,Charlie Cox,Claire Danes,Sienna Miller,Ian McKellen,255036,"38,634,938","[0.018057307228446007, -0.030250301584601402, ...",0.154631
778,https://m.media-amazon.com/images/M/MV5BNjk1Nz...,The Machinist,2004,R,101 min,"Drama, Thriller",7.7,An industrial worker who hasn't slept in a yea...,61.0,Brad Anderson,Christian Bale,Jennifer Jason Leigh,Aitana Sánchez-Gijón,John Sharian,358432,"1,082,715","[0.010055247694253922, -0.007203566376119852, ...",0.157752


Great, now we have the dataset sorted from most relevant to least relevant! Let's save this as a CSV so we can load it in the next step and compose a custom prompt.

Run the cell below as-is.

In [13]:
df.to_csv("distances.csv")

# Steps 3 & 4: Querying a Completion Model with a Custom Text Prompt

Add your API key to the cell below then run it.

In [15]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = OPENAI_API_KEY

The code below loads in the data sorted by cosine distance that you previously created. Run it as-is.

In [37]:
import pandas as pd

df = pd.read_csv("distances.csv", index_col=0)
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,embeddings,distances
8,https://m.media-amazon.com/images/M/MV5BMjAxMz...,Inception,2010,UA,148 min,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,2067042,"292,576,195",[-0.00540503 -0.03797875 0.00259735 ... 0.01...,0.117507
643,https://m.media-amazon.com/images/M/MV5BMWM0Zj...,Waking Life,2001,R,99 min,"Animation, Drama, Fantasy",7.8,A man shuffles through a dream meeting various...,83.0,Richard Linklater,Ethan Hawke,Trevor Jack Brooks,Lorelei Linklater,Wiley Wiggins,60684,"2,892,011",[ 0.00906853 -0.02517219 0.01816321 ... -0.02...,0.131207
959,https://m.media-amazon.com/images/M/MV5BMGExOG...,Dark City,1998,A,100 min,"Mystery, Sci-Fi, Thriller",7.6,"A man struggles with memories of his past, whi...",66.0,Alex Proyas,Rufus Sewell,Kiefer Sutherland,Jennifer Connelly,William Hurt,187927,"14,378,331",[ 0.00564287 -0.00832251 0.01134946 ... 0.00...,0.143791
919,https://m.media-amazon.com/images/M/MV5BMjkyMT...,Stardust,2007,U,127 min,"Adventure, Family, Fantasy",7.6,In a countryside town bordering on a magical l...,66.0,Matthew Vaughn,Charlie Cox,Claire Danes,Sienna Miller,Ian McKellen,255036,"38,634,938",[ 0.01805731 -0.0302503 -0.00038912 ... -0.00...,0.154631
778,https://m.media-amazon.com/images/M/MV5BNjk1Nz...,The Machinist,2004,R,101 min,"Drama, Thriller",7.7,An industrial worker who hasn't slept in a yea...,61.0,Brad Anderson,Christian Bale,Jennifer Jason Leigh,Aitana Sánchez-Gijón,John Sharian,358432,"1,082,715",[ 0.01005525 -0.00720357 0.00296646 ... -0.00...,0.157752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,https://m.media-amazon.com/images/M/MV5BMjE0Nz...,Letters from Iwo Jima,2006,UA,141 min,"Action, Adventure, Drama",7.9,The story of the battle of Iwo Jima between th...,89.0,Clint Eastwood,Ken Watanabe,Kazunari Ninomiya,Tsuyoshi Ihara,Ryô Kase,154011,"13,756,082",[-0.01596861 -0.01751789 -0.00678879 ... -0.00...,0.270159
182,https://m.media-amazon.com/images/M/MV5BNDc2OD...,Judgment at Nuremberg,1961,A,179 min,"Drama, War",8.2,"In 1948, an American court in occupied Germany...",60.0,Stanley Kramer,Spencer Tracy,Burt Lancaster,Richard Widmark,Marlene Dietrich,69458,NaN,[ 0.00207382 -0.02525822 -0.00124879 ... 0.01...,0.270688
606,https://m.media-amazon.com/images/M/MV5BMTA5Mz...,Straight Outta Compton,2015,R,147 min,"Biography, Drama, History",7.8,The rap group NWA emerges from the mean street...,72.0,F. Gary Gray,O'Shea Jackson Jr.,Corey Hawkins,Jason Mitchell,Neil Brown Jr.,179264,"161,197,785",[ 3.44860787e-03 -2.86256596e-02 -1.06840469e-...,0.270982
462,https://m.media-amazon.com/images/M/MV5BMTEyMT...,Bronenosets Potemkin,1925,NaN,75 min,"Drama, History, Thriller",8.0,In the midst of the Russian Revolution of 1905...,97.0,Sergei M. Eisenstein,Aleksandr Antonov,Vladimir Barskiy,Grigoriy Aleksandrov,Ivan Bobrov,53054,"50,970",[-0.02347166 -0.03147098 -0.01894574 ... 0.02...,0.272988


## TODO 1: Build the Custom Text Prompt

Run the cell below as-is:

In [22]:
!pip install tiktoken
import tiktoken
# Create a tokenizer that is designed to align with our embeddings
tokenizer = tiktoken.get_encoding("cl100k_base")

token_limit = 1000
USER_QUESTION = """A man is stuck in a dream world where time moves differently, and he has to plant an idea in someone's mind."""

   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   --------------------------------------- 894.7/894.7 kB 16.8 MB/s eta 0:00:00


In [ ]:
# get the first 3 movies names and descriptions
df_3movies = df.head(3)
# save them in a list of json objects
df_3movies_list = df_3movies[["Series_Title", "Overview"]].to_dict(orient="records")
#  then dump them in string format
import json
df_3movies_list_str = json.dumps(df_3movies_list)
# get the number of tokens in the string
num_tokens = len(tokenizer.encode(df_3movies_list_str))
print(f"Number of tokens in the string: {num_tokens}")
# print the string
print(df_3movies_list_str)
# combine the df_3movies_list_str in a single string called context
prompt_template = f"""You are a movie recommendation system. You will be given a list of movies and their descriptions. Based on the descriptions, you will recommend the best movie that matches the user's question.
The user question is: {USER_QUESTION}  
The list of movies and their descriptions is: {df_3movies_list_str}
you answer the question by giving the name of the movie and a short description of it.
Answer:"""
# get the number of tokens in the context string
num_tokens = len(tokenizer.encode(context))
print(f"Number of tokens in the context string: {num_tokens}")
print(prompt_template)

Number of tokens in the string: 124
[{"Series_Title": "Inception", "Overview": "A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O."}, {"Series_Title": "Waking Life", "Overview": "A man shuffles through a dream meeting various people and discussing the meanings and purposes of the universe."}, {"Series_Title": "Dark City", "Overview": "A man struggles with memories of his past, which include a wife he cannot remember and a nightmarish world no one else ever seems to wake up from."}]
Number of tokens in the context string: 200
You are a movie recommendation system. You will be given a list of movies and their descriptions. Based on the descriptions, you will recommend the best movie that matches the user's question.
The user question is: A man is stuck in a dream world where time moves differently, and he has to plant an idea in someone's mind.  
The list of movies and their descriptions

In [32]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=prompt_template,
    max_tokens=350
)
answer = response["choices"][0]["text"].strip()
print(answer)

The recommended movie is "Inception" where a man is stuck in a dream world and has to plant an idea in someone's mind to complete his mission. In this thrilling and mind-bending movie, the main character enters the dream world through advanced technology and faces challenges as he tries to manipulate the dreams of others. The concept of time being distorted in dreams adds an additional layer of suspense to the story.


In [44]:
movies = [
    {
        "description": "A man is stuck in a dream world where time moves differently, and he has to plant an idea in someone's mind.",
        "movie": "Inception"
    },
    {
        "description": "It’s about a prisoner who never loses hope and secretly works on escaping for years, all while forming a deep friendship with another inmate.",
        "movie": "The Shawshank Redemption"
    },
    {
        "description": "A guy dressed like a bat fights a crazy clown who just wants to watch the world burn.",
        "movie": "The Dark Knight"
    },
    {
        "description": "Two hitmen talk about random stuff while working for a gangster, and there's a scene with a dance contest at a diner.",
        "movie": "Pulp Fiction"
    },
    {
        "description": "A soldier keeps living the same brutal day during an alien invasion, resetting every time he dies.",
        "movie": "Edge of Tomorrow"
    },
    {
        "description": "A boy wizard discovers he's famous in a magical world and goes to a castle-like school, where he makes two best friends.",
        "movie": "Harry Potter and the Sorcerer’s Stone"
    },
    {
        "description": "A guy falls in love with an AI voice assistant that sounds super real, and their relationship gets complicated. ",
        "movie": "Her"
    },
    {
        "description": "This animated movie is about a rat who wants to become a chef in Paris. He hides under a guy’s ",
        "movie": "Ratatouille"
    },
    {
        "description": "A team of scientists gets stuck inside a park where dinosaurs come back to life and everything goes wrong. There's a lot of running and screaming with a T-Rex.",
        "movie": "Jurassic Park"
    },
    {
        "description": "A king’s son thinks he’s responsible for his father’s death and runs away, but later returns to face his evil uncle. There's a lot of singing animals.",
        "movie": "The Lion King"
    }
]

import numpy as np
import pandas as pd

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)


for movie in movies:
    # Update the user question with the movie description
    USER_QUESTION = movie["description"]
    
    # Generate the embedding for the updated question
    # Generate the embedding response
    response = openai.Embedding.create(
        input=USER_QUESTION,
        engine=EMBEDDING_MODEL_NAME
    )

# Extract the embeddings from the response
    question_embeddings = response["data"][0]["embedding"]  
    
    
    # Calculate distances between the question embedding and dataset embeddings
    distances = distances_from_embeddings(
        question_embeddings,
        df["embeddings"],
        distance_metric="cosine"
    )
    
    # Add distances to the dataframe and sort by relevance
    df["distances"] = distances
    df.sort_values(by="distances", ascending=True, inplace=True)
    
    # Get the top 3 most relevant movies
    df_3movies = df.head(3)
    df_3movies_list = df_3movies[["Series_Title", "Overview"]].to_dict(orient="records")
    df_3movies_list_str = json.dumps(df_3movies_list)
    
    # Create the prompt
    prompt_template = f"""You are a movie recommendation system. You will be given a list of movies and their descriptions. Based on the descriptions, you will recommend the best movie that matches the user's question.
    The user question is: {USER_QUESTION}  
    The list of movies and their descriptions is: {df_3movies_list_str}
    you answer the question by giving the name of the movie and a short description of it.
    Answer:"""
    
    # Generate the response from the chatbot
    response = openai.Completion.create(
        model=COMPLETION_MODEL_NAME,
        prompt=prompt_template,
        max_tokens=1000
    )
    answer = response["choices"][0]["text"].strip()
    
    # Print the evaluation result
    print(f"User Question: {USER_QUESTION}")
    print(f"Chatbot Answer: {answer}")
    print(f"Expected Movie: {movie['movie']}")
    print("-" * 50)

User Question: A man is stuck in a dream world where time moves differently, and he has to plant an idea in someone's mind.
Chatbot Answer: Inception - A thief has to plant an idea in someone's mind in a dream world where time moves differently.
Expected Movie: Inception
--------------------------------------------------
User Question: It’s about a prisoner who never loses hope and secretly works on escaping for years, all while forming a deep friendship with another inmate.
Chatbot Answer: The Shawshank Redemption - Two imprisoned men form a deep friendship while one of them secretly works on escaping for years.
Expected Movie: The Shawshank Redemption
--------------------------------------------------
User Question: A guy dressed like a bat fights a crazy clown who just wants to watch the world burn.
Chatbot Answer: The Dark Knight: "When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tes